In [3]:
#Reading the HFRAdmin file
import pandas as pd
import numpy as np

data=pd.read_csv("/Users/Tarek/ml/datasets/HFs/Dataset-HFRAdmin.csv")


In [29]:

#example of customizing the tokenizer in sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

#Using NLTK
from nltk.stem.snowball import SnowballStemmer
from nltk import WordNetLemmatizer
stemmer=SnowballStemmer("english")
wnl=WordNetLemmatizer()

X=data['Strategy_Description']
y=data.Main_Strategy + ' > ' + data.Sub_Strategy
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.33,random_state=42)

def custom_normalizer(tokens):
    """Lemmatization, Stemming and Map numeric tokens to a placeholder.
    
    This is useful to use when numbers are not material 
    in our machine learning task. Lemmatization and stemming ensure that
    words of the same meaning do not appear as seperate features"""
    
    for t in tokens:
        if t[0].isdigit():
            t='#NUMBER'
        else:
            t=wnl.lemmatize(t)
            t=stemmer.stem(t)
            #t=t
        yield t

class CustomVectorizer(TfidfVectorizer):
    def build_tokenizer(self):
        tokenize=super(CustomVectorizer,self).build_tokenizer()
        return lambda doc: list(custom_normalizer(tokenize(doc)))

    
v=CustomVectorizer(stop_words='english',norm='l2',sublinear_tf=False,max_df=.5)
X1=v.fit_transform(X_train)
X1

<6424x7218 sparse matrix of type '<class 'numpy.float64'>'
	with 335843 stored elements in Compressed Sparse Row format>

In [32]:
y_test[0]

'Relative Value > Fixed Income - Asset Backed'

In [34]:
X_test[0]

'The Fund aims to generate positive returns by taking advantage of the Quantitative Easing (QE) cycle using the duration and volatility characteristics of Interest Only securities (IOs), a class of mortgage backed securities.'

In [36]:
from sklearn.naive_bayes import MultinomialNB

NBclf=MultinomialNB().fit(X1,y_train)

In [47]:
#testing 
X2=v.transform(X_test[0:2])
prediction=NBclf.predict(X2)

prediction

array(['Equity Hedge > Fundamental Value',
       'Equity Hedge > Fundamental Value'],
      dtype='<U59')

In [39]:
y_test[0:2]

5145    Equity Hedge > Fundamental Value
4230     Relative Value > Multi-Strategy
dtype: object

In [56]:
from sklearn.pipeline import Pipeline

NBPipeline=Pipeline([('vectorize',CustomVectorizer(stop_words='english',norm='l2',sublinear_tf=False,max_df=.5)),('NBClassify',MultinomialNB())])

NBPipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vectorize', CustomVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=0.5, max_features=None, min_df=1,
         ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf...      vocabulary=None)), ('NBClassify', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [57]:
import numpy as np

predicted=NBPipeline.predict(X_test)

np.mean(predicted==y_test)

0.34723538704581358